# Excercise 1.4

Use the pySINDY algorithm to find the best-fit nonlinear model to describe the data

We install some usefull libraries

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.linear_model import Lasso
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error
from scipy.integrate import solve_ivp

#git clone https://github.com/dynamicslab/pysindy.git
 #   pip install .
!pip install pysindy
import pysindy as ps
# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Seed the random number generators for reproducibility
np.random.seed(100)



We define our data

We interpolate the data with a spline function, it helps stabilizing the algorithm

In [2]:
hare = [20, 20, 52, 83, 64, 68, 83, 12, 36, 150, 110, 60, 7, 10, 70, 100, 92, 70, 10, 11, 137, 137, 18, 22, 52, 83, 18, 10, 9, 65]
lynx = [32, 50, 12, 10, 13, 36, 15, 12, 6 ,6, 65, 70, 40, 9, 20, 34, 45, 40, 15, 15, 60, 80, 26, 18, 37, 50, 35, 12, 12, 25]
t0 = 1845;
t1 = 1903;
dt = 2;
t_series = np.arange(t0,t1+dt,dt)-t0
X = np.stack((hare,lynx),axis=-1)
print(np.shape(t_series))
print(np.shape(X))

import scipy
Sp_i = scipy.interpolate.CubicSpline(t_series,X,axis=0)

dt_test = 0.01
t_test = np.arange(t0,t1,dt_test)-t0
X_s = Sp_i(t_test)

plt.figure()
plt.plot(t_test+t0,X_s[:,0],color='b',linestyle='dashed',label='hare SINDy')
plt.plot(t_test+t0,X_s[:,1],color='r',linestyle='dashed',label='lynx SINDy')
plt.plot(t_series+t0,X[:,0],color='b',linestyle='solid',label='hare')
plt.plot(t_series+t0,X[:,1],color='r',linestyle='solid',label='lynx')
plt.grid()
plt.legend(fontsize=12)
plt.show()

The first non-linear model is fit

In [3]:
# the equations I need to fit are:
# x'=(b-py)x and y'=(rx-d)y
# x' = bx-p*xy and y' = r*xy-dy

# MODEL 0
feat_names = ['hare','lynx']
feat_library = ps.PolynomialLibrary(degree=3)
optimizer = ps.STLSQ(threshold=0.0001,max_iter=100)
differentiation_method = ps.FiniteDifference(order=3,drop_endpoints=False)
#differentiation_method = ps.SmoothedFiniteDifference()
#differentiation_method = ps.SINDyDerivative(kind="spline", s=1e-2)
model0 = ps.SINDy(differentiation_method = differentiation_method,
            feature_names=feat_names,
            optimizer=optimizer,
            feature_library = feat_library)


model0.fit(X_s,t=t_test)
model0.print()
print("Feature names:\n", model0.get_feature_names())
integrator_keywords = {}
#integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'RK45'#'LSODA' 'Radau'
integrator_keywords['max_step'] = 100
integrator_keywords['min_step'] = 1
#integrator_keywords['atol'] = 1e-12

We compare the original data with the results for the firs model

In [4]:
dt_test = 0.01
t_test = np.arange(t0,t1,dt_test)-t0
X0 = X[0,:]
sim = model0.simulate(X0,t_test, integrator_kws=integrator_keywords)

plt.figure()
plt.plot(t_test+t0,sim[:,0],color='b',linestyle='dashed',label='hare SINDy')
plt.plot(t_test+t0,sim[:,1],color='r',linestyle='dashed',label='lynx SINDy')
plt.plot(t_series+t0,X[:,0],color='b',linestyle='solid',label='hare')
plt.plot(t_series+t0,X[:,1],color='r',linestyle='solid',label='lynx')
plt.grid()
plt.legend(fontsize=12)
plt.show()

# Bagging

We perform bagging to see if we can improve the model

At first we fit 10000 models with a reduced threshold

In [5]:

ensemble_optimizer = ps.STLSQ(threshold=0.0001,max_iter=100)
library = ps.PolynomialLibrary(degree=3)
model_1 = ps.SINDy(feature_names=feat_names,
                  optimizer = ensemble_optimizer,
                  differentiation_method=differentiation_method,
                  feature_library = library)
model_1.fit(X_s, t=t_test, ensemble=True, quiet=True,replace=True, n_models = 10000,n_subset=24) 
model_1.print()

ensemble_coefs = (model_1.coef_list)

print(np.shape(ensemble_coefs))
print(model_1.get_feature_names())

Make plots of each coefficient distribution!

In [6]:
plt.figure(figsize=(20, 20))

ensemble_coefs_1 = np.asarray(ensemble_coefs)

for j in range(10):
    for i in range(2):
        plt.subplot(10, 2, i + 1 + j * 2)
        if j == 0:
            plt.title(feat_names[i], fontsize=14)
        coef_mean = np.mean(ensemble_coefs_1[:,i,j])
        coef_std = np.std(ensemble_coefs_1[:,i,j])
        bins = np.linspace(coef_mean-2*coef_std,coef_mean+2*coef_std, 51)#
        plt.hist(ensemble_coefs_1[:, i, j], color='b', bins=bins, 
                 label='ensemble', align='left')

        plt.grid(True)
        ax = plt.gca()
        if i == 0:
            xticknames = model_1.get_feature_names()
            plt.ylabel(xticknames[j], fontsize=12)
        else:
            ax.set_yticklabels([])
        plt.ylim(0, 2000)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        if i == 2 and j == 9:
            plt.legend(fontsize=10)
            
#plt.savefig('file_name.png')

Now we try to change slightly the thresholds for the differnet coefficients 

In [7]:
library = ps.PolynomialLibrary(degree=3)
library.fit(X)
n_features = library.n_output_features_
thresholds = np.ones((n_features,2))*100
thresholds[0,:] = .001
thresholds[1,:] = 0.001
thresholds [2,:] = 0.001
thresholds [3,0] = 0.001
#thresholds[3,:] = 0.0001
#thresholds[4,:] = 0.0001#0
#thresholds[5,:] = 0.0001
#thresholds[6,0] = 0.0001
#thresholds[7,0] = 0.0001
thresholds[8,0] = 0.0001
thresholds[9,0] = 0.0001

In [8]:
thresholds = np.ones((n_features,2))*100
thresholds[0,:] = .01
thresholds[1,:] = 0.001
thresholds [2,:] = 0.001
#thresholds [3,0] = 0.001
thresholds[3,:] = 0.001
thresholds[4,:] = 0.0001#0
thresholds[5,:] = 0.0001
thresholds[6,:] = 0.0001
thresholds[7,:] = 0.0001
thresholds[8,:] = 0.0001
thresholds[9,:] = 0.0001

We fit again 10000 models 

In [9]:
sr3_optimizer = ps.SR3(thresholder="weighted_l0", thresholds=thresholds,max_iter=100)
model_4 = ps.SINDy(feature_names=feat_names,
                  optimizer = sr3_optimizer,
                  differentiation_method=differentiation_method,
                  feature_library = library)
n_models = 10000
model_4.fit(X_s, t=t_test, ensemble=True, quiet=True,replace=True, n_models = n_models,n_subset=24) 
model_4.print()

w_ensemble_coefs = (model_4.coef_list)

print(np.shape(w_ensemble_coefs))
print(model_4.get_feature_names())

We plot again the distribution of the coefficients

In [10]:
plt.figure(figsize=(20, 20))

w_ensemble_coefs_1 = np.asarray(w_ensemble_coefs)

for j in range(10):
    for i in range(2):
        plt.subplot(10, 2, i + 1 + j * 2)
        if j == 0:
            plt.title(feat_names[i], fontsize=14)
        coef_mean = np.mean(w_ensemble_coefs_1[:,i,j])
        coef_std = np.std(w_ensemble_coefs_1[:,i,j])
        bins = np.linspace(coef_mean-2*coef_std,coef_mean+2*coef_std, 51)#
        plt.hist(w_ensemble_coefs_1[:, i, j], color='b', bins=bins, 
                 label='ensemble', align='left')

        plt.grid(True)
        ax = plt.gca()
        if i == 0:
            xticknames = model_4.get_feature_names()
            plt.ylabel(xticknames[j], fontsize=12)
        else:
            ax.set_yticklabels([])

        plt.ylim(0, 500)

        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        if i == 2 and j == 9:
            plt.legend(fontsize=10)
            
#plt.savefig('thres_w.png')

We simulate and plot randomly one of the models

In [11]:
w_ensemble_coefs_1.shape[0]
i0 = np.random.randint(0,n_models)
print(i0)
#i0=261
sr3_optimizer.coef_ = w_ensemble_coefs_1[i0,:,:]
print( w_ensemble_coefs_1[i0,:,:])
integrator_keywords = {}
#integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'RK45'#'LSODA' 'RK45'
integrator_keywords['max_step'] = 100
integrator_keywords['min_step'] = 1
x_test_i = model_4.simulate(X0,t_test,integrator='odeint')#,integrator_kws=integrator_keywords)integrator='odeint'
print(np.shape(x_test_i))
if np.shape(x_test_i)[0] == 5800:
    plt.figure()
    plt.plot(t_test+t0,x_test_i[:,0],color='b',linestyle='dashed',label='hare SINDy')
    plt.plot(t_test+t0,x_test_i[:,1],color='r',linestyle='dashed',label='lynx SINDy')
    plt.plot(t_series+t0,X[:,0],color='b',linestyle='solid',label='hare')
    plt.plot(t_series+t0,X[:,1],color='r',linestyle='solid',label='lynx')
    plt.grid()
    plt.legend(fontsize=12)
    plt.show()

print([(np.var(x_test_i[-1000:-1,0]) , np.var(x_test_i[-1000:-1,1]) )])

We simulate all of the models in order to exclude the unstable ones and the ones that saturate in time

In [12]:
x_test_4 = np.zeros([n_models,5800,2])
X0 = X[1]
ns = 0
nt = 0
coef_stable = []
coef_unstable = []
x_test_i = np.zeros((2,5800))
x_test_stable =  np.zeros([n_models,5800,2])
coef_unstable = np.zeros([n_models,2,10])
coef_stable = np.zeros([n_models,2,10])

for i in range(w_ensemble_coefs_1.shape[0]):
    sr3_optimizer.coef_ = w_ensemble_coefs_1[i,:,:]
    x_test_i = model_4.simulate(X0,t_test,integrator = 'odeint')#,integrator_kws=integrator_keywords)

    if np.shape(x_test_i)[0] == 5800:
        x_test_4[i,:,:] = x_test_i
        if np.any(np.abs(x_test_i) > 400) or (np.var(x_test_i[-1000:-1,0])< 1 or np.var(x_test_i[-1000:-1,1]) < 1):

            coef_unstable[ns,:,:] = w_ensemble_coefs_1[i,:,:]
            ns =ns+1
        else:
            coef_stable[nt,:,:] = (w_ensemble_coefs_1[i,:,:])
            x_test_stable[nt,:,:] = (x_test_i)
            nt =nt+1
    else:
        coef_unstable[ns,:,:] = w_ensemble_coefs_1[i,:,:]
        ns =ns+1 
    
print(np.shape(coef_stable))
print([ns,'unstable models out of ',n_models])

We simulate the stable models

In [13]:
coef_stable_1 = coef_stable[0:(n_models-ns),:,:]
x_test_stable = np.zeros([nt,5800,2])
X0 = X[1]

for i in range(nt):
    sr3_optimizer.coef_ = coef_stable_1[i,:,:]

    x_test_stable[i,:,:] = model_4.simulate(X0,t_test,integrator = 'odeint')
    
mean_x_stable = np.mean(x_test_stable,axis=0)
plt.figure()
plt.plot(t_test+t0,mean_x_stable[:,0],color='b',linestyle='dashed',label='hare SINDy')
plt.plot(t_test+t0,mean_x_stable[:,1],color='r',linestyle='dashed',label='lynx SINDy')
plt.plot(t_series+t0,X[:,0],color='b',linestyle='solid',label='hare')
plt.plot(t_series+t0,X[:,1],color='r',linestyle='solid',label='lynx')
plt.grid()
plt.legend(fontsize=12)
plt.show()    
#print(np.shape(coef_stable))
#print([ns,'unstable models out of ',n_models])

We plot the distributions of the coefficients of the models we selected 

In [14]:
plt.figure(figsize=(20, 20))

for j in range(10):
    for i in range(2):
        plt.subplot(10, 2, i + 1 + j * 2)
        if j == 0:
            plt.title(feat_names[i], fontsize=14)
        coef_mean = np.mean(coef_stable_1[:,i,j])
        coef_std = np.std(coef_stable_1[:,i,j])
        bins = np.linspace(coef_mean-2*coef_std,coef_mean+2*coef_std, 51)#
        plt.hist(coef_stable_1[:, i, j], color='b', bins=bins, 
                 label='ensemble', align='left')

        plt.grid(True)
        ax = plt.gca()
        if i == 0:
            xticknames = model_4.get_feature_names()
            plt.ylabel(xticknames[j], fontsize=12)
        else:
            ax.set_yticklabels([])

        plt.ylim(0, 50)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        if i == 2 and j == 9:
            plt.legend(fontsize=10)
            
#plt.savefig('thres_w.png')

In [15]:
np.shape(coef_stable_1)

We calculate the mean values of the coefficients and use them to make the prediction

In [16]:
coef_stable_1 = coef_stable[0:(n_models-ns),:,:]
cosef_mean = np.mean(coef_stable_1,axis=0)
sr3_optimizer.coef_ = cosef_mean
x_test_mean = model_4.simulate(X0,t_test,integrator = 'odeint')

dt_test = 0.01
t_test = np.arange(t0,t1,dt_test)-t0
X0 = X[0,:]
sim = model0.simulate(X0,t_test, integrator_kws=integrator_keywords)

plt.figure()
plt.plot(t_test+t0,x_test_mean[:,0],color='b',linestyle='dashed',label='hare SINDy')
plt.plot(t_test+t0,x_test_mean[:,1],color='r',linestyle='dashed',label='lynx SINDy')
plt.plot(t_series+t0,X[:,0],color='b',linestyle='solid',label='hare')
plt.plot(t_series+t0,X[:,1],color='r',linestyle='solid',label='lynx')
plt.grid()
plt.legend(fontsize=12)
plt.show()